**To use the examples in this chapter, first run the code below to import the right libraries.**

This is the first time we introduce the PyCh simulation library.

In [3]:
# =================================
# Imports
# =================================
from PyCh import *
from numpy import random
from dataclasses import dataclass
import math

PyCh version 0.4 imported succesfully.
 


# 7 Processes

The PyCh simulation library has been designed for modeling and analyzing systems with many components, all working together to obtain the total system behavior. Each component exhibits behavior over time. Sometimes they are busy making internal decisions, sometimes they interact with other components. The language uses a process to model the behavior of a component (the primary interest are the actions of the component rather than its physical representation). This leads to models with many processes working in parallel (also known as concurrent processes), interacting with each other.

Another characteristic of these systems is that the parallelism happens at different scales at the same time, and each scale can be considered to be a collection of co-operating parallel working processes. For example, a factory can be seen as a single component, it accepts supplies and delivers products. However, within a factory, you can have several parallel operating production lines, and a line consists of several parallel operating machines. A machine again consists of parallel operating parts. In the other direction, a factory is a small element in a supply chain. Each supply chain is an element in a (distribution) network. Depending on the area that needs to be analyzed, and the level of detail, some scales are precisely modeled, while others either fall outside the scope of the system or are modeled in an abstract way.

In all these systems, the interaction between processes is not random, they understand each other and exchange information. In other words, they communicate with each other. PyCh uses channels to model the communication. A channel connects a sending process to a receiving process, allowing the sender to pass messages to the receiver. This chapter discusses parallel operating processes only, communication between processes using channels is discussed in Chapter 8.

As discussed above, a process can be seen as a single component with behavior over time, or as a wrapper around many processes that work at a smaller scale. PyCh supports both kinds of processes.

## 7.1 The simulation environment

When we create a model in PyCh, we must first define the simulation environment in which the processes of this model live. We typically do so by starting our model with: `env = Environment()`, which creates instance `env` of the `Environment` class. After creating an environment, the next step is to define the processes living in this environment (more on this in the next section). When all processes have been defined, we can simulate an instance of the `Environment` class by calling its `run()` function, so for environment `env` this is done through `env.run()`. When we run an environment, the processes in the environment start running. The simulation stops when all processes have finished executing.

Below is an example of how we generally start when creating a model. We can execute this model, but nothing will happen, as our environment does not yet contain any processes.

In [ ]:
def model():  # we define our model
    env = Environment()  # we define its simulation environment

    '''
    The processes in this simulation environment go here
    '''

    env.run() # we run the simulation environment

model() # we execute our model

### 7.1.1 Simulation time
When we talk about the simulation environment, we must also explain the concept of *simulation time*. With simulation time, we mean the time inside the simulation environment. This is not the same as real-time! A simulation of a lengthy process spanning days, months or even years in simulation time, could only take a few seconds to execute in real-time! In our models, the current simulation time can be accessed through the `Environment.now` variable.

When we run a simulation, the default option is to run until all processes have finished executing. However, we can also run our environment until simulation time `t` using `Environment.run(until=t)`, or we can run it until a process `P` has finished executing using `Environment.run(until=P)`. How we define such a process is explained in the next section.

The example below shows for environment `env` how `env.run(until=t)` and `env.now` can be utilized. Again, there are no processes, so the simulation ends without anything happening after 5 units of simulation time. Note: these units of simulation time are whatever you define them to be. One unit can represent 1 second, but it can also represent 15 minutes, or 7 days, or whatever you want it to.

In [ ]:
def model(): 
    env = Environment() 

    '''
    The processes in this simulation environment go here
    '''

    env.run(until=5) # we run the simulation environment until time = 5
    print(f"The simulation ends after {env.now} days.")  # we print the simulation time

model()

## 7.2 A single process
After defining the simulation environment, the next step is to define the processes living in this environment. In Pych processes are defined using process functions, which are denoted by the decorator `@process` above the function definition. A process function must always pass as the simulation environment `env` in which the process lives as one of its arguments.

A Pych process can generate events and can `yield` these events. The `yield` statement indicates that the process is suspended till the event has been triggered, after which the process continues. A process must always yield at least one event, so in the next subsection we will introduce the first type of event that will be used.

### 7.2.1 The timeout event
The first event type that we introduce is the `Environment.timeout(t)` event. This event denotes a timeout of `t` in the environment's simulation time. 

Below we extend our model with its first process. 
We define process function `P` with the simulation environment `env` as its only argument.
Process `P` contains three statements.
The first is `delay = env.timeout(2.5)`, which indicates that we create a `delay` event that will trigger after `2.5` units in simulation time. The second is `yield delay`, which indicates that the process waits till the `delay` event has triggered,
 after which it continues.
The third statement is the `print` statement, in which we use the `env.now` function to show what the current simulation time is. 

The model below contains one instance (`P1`) of the process function (`P`).

In [ ]:
@process
def P(env):
    delay = env.timeout(2.5)  # we define the timeout event
    yield delay  # The process waits till the timeout event is triggered
    print(f"Hello. I am a process which is finished after {env.now} hours.")

def model():
    env = Environment()
    P1 = P(env)  # Our model has one instance of process P
    env.run()
    
model()

---
A model environment can contain multiple instances created using the same process definition. When simulating the environment using `env.run()`, the simulation continues until both processes are finished executing. To demonstrate, below is an example of a model with two processes `P1` and `P2`. 

In the same example, we also show with `yield env.timeout(1)` that we can create an event, and yield it on the same line.

In [ ]:
@process
def P(env, i):
    yield env.timeout(1)  # we create an event and yield it on the same line
    print(f"I am process {i}")

def model():
    env = Environment()
    P1 = P(env, 1)  # our first instance of process P
    P2 = P(env, 2)  # our second instance of process P
    env.run()
    
model()

--- 
Note that when we define a timeout event, its timer starts ticking as soon as the event is defined.
The `yield` statement indicates that the process waits till the event has been triggered. If the event had already started earlier, or has already been triggered, then the process does not wait the entire timeout duration.
In the example below we can see how this works in practice.

As you can see in the example, the clock of the timeout event starts ticking when the event is defined. When Process `P` encounters the first yield statement it is suspended till the timeout event `delay1` is triggered at `t=0.5`, then it is suspended again till `delay2` is triggered at `t=1.0` (so not at `t=1.5`!). Finally, at the third yield statement, process `P` does not suspend, instead it continues right away, because event `delay3` had already been triggered in the past.


In [ ]:
@process
def P(env):
    delay1 = env.timeout(0.5)  # this event is triggered after 0.5 seconds
    delay2 = env.timeout(1.0)  # this event is triggered after 1.0 seconds
    delay3 = env.timeout(0.5)  # this event is also triggered after 0.5 seconds
    yield delay1
    print(f"The first timeout event was triggered at time {env.now:.1f}")
    yield delay2
    print(f"The second timeout event was triggered at time {env.now:.1f}")
    yield delay3
    print(f"The third timeout event had already been triggered, so the process continues right away at time {env.now:.1f}")

def model():
    env = Environment()
    P1 = P(env)
    env.run()

model()


## 7.3 Process in process

A process can create other processes, and it can even wait until another process has finished through the `yield` statement (although it can also continue unhalted). In the example below main process `P` creates sub processes `P1` and `P2`, and waits till they have finished executing to continue. The concept of 'a process in a process' is very useful in keeping the model structured.

In [ ]:
@process
def main_process(env):
    print("Start process 1")
    P1 = sub_process(env, 1)  # we start process P1
    yield P1  # we wait untill process P1 has finished executing
    print("Start process 2")
    P2 = sub_process(env, 2)  # we start process P2
    yield P2  # we wait untill process P2 has finished executing
    
@process
def sub_process(env, i):
    yield env.timeout(1)
    print(f"Finished process {i}")
    
def model():
    env = Environment()
    P = main_process(env)
    env.run()
    
model()

---
Just like with the timeout event, a process starts running when it is defined, not when we yield it. Suppose we redefine the main process as seen below. If we run our model again, we can see that the order of execution is changed. 

In [ ]:
@process
def main_process(env):
    print("Start process 1")
    P1 = sub_process(env, 1) # we start process P1
    print("Start process 2")
    P2 = sub_process(env, 2) # we start process P2
    yield P1 # we wait untill process P1 has finished executing
    yield P2 # we wait untill process P2 has finished executing
    
model()

## 7.4 Many processes

Some models consist of many similar processes. In Python, we can utilize list comprehension to quickly create many processes. Below is an example of using list comprehension to create 10 instances of `P` at once.

In [ ]:
@process
def P(env, i):
    yield env.timeout(1)
    print(f"I am process {i}")

def model():
    env = Environment()
    Processes = [P(env, i) for i in range(10)]  # we use list comprehension to create 10 instances of P
    env.run()
    
model()

## 7.5 Process output

A process can `return` an output (e.g. `return main_output` and `return sub_output` in the example below). This can be utilized in two ways.

Firstly, when one process yields another process, it can use the output of that other process. For example, `sub_output = yield P1` in the example below. Alternatively, we can access the Process' property `Process.value` to access its output (e.g. `sub_output_alternative = P1.value` in the example below). Both methods result in the same output.

Secondly, we can use `Process.value` to retrieve output from our process after the model has finished simulating. For example, `main_output = P.value` in the example below. This can be very useful when we want to retrieve information from our processes after our model has finished executing (e.g. the throughput of a machine process).

In [10]:
@process
def main_process(env):
    P1 = sub_process(env)
    sub_output = yield P1
    sub_output_alternative = P1.value
    print(f"Our subprocess returns {sub_output}, or using the alternative method, also: {sub_output_alternative}.")
    main_output = sub_output*2
    return main_output
    
@process
def sub_process(env):
    yield env.timeout(1)
    sub_output = 1
    return sub_output
    
def model():
    env = Environment()
    P = main_process(env)
    env.run()
    main_output = P.value
    print(f"The main process returns {main_output}")
    
model()

Our subprocess returns 1, or using the alternative method, also: 1
The main process returns 2


## 7.5 Summary
- A process represents the behavior of a component in the simulation environment. An environment can have multiple processes running in parallel.
- The simulation environment is defined using `env = Environment()`. 
    - The simulation environment can be run until all of its processes have finished executing using `env.run()`.
    - The simulation environment can be run until simulation time `t` using `env.run(until=t)`.
    - The simulation environment can be run until one of its processes has finished executing using `env.run(until=Process)`.
    - The current simulation time can be accessed using `env.now`.
- A process function specified with the `@process` decorator above it, and it is defined using `def P(env, ...)` with its simulation environment as its first argument.
    - A process can be instantiated by calling the process function, e.g. `P1 = P(env, ...)`.
    - Multiple processes can be instantiated from the same process function, e.g. `P1 = P(env, 1)` and `P2 = P(env, 2)`.
    - Many similar processes can be instantiated using list comprehension, e.g. `Processes = [P(env, i) for i in range(10)]`.
    - A process can instantiate other processes.
- A process can create and `yield` events through its lifetime.
    - One type of event is the timeout event `timeout_event = env.timeout(t)`, which is an event that is triggered after `t` simulation time. 
    - Other events are communication events, which are explained in the next chapter.
    - A process can yield these events through `yield Event`, which means that the process will be suspended until the event has been triggered, after which the process continues.
    - A process can yield other processes through `yield Process`, which means that the process will be suspended until the other process has finished executing.
- A process can `return` an output when it has finished.
    - This output can be retrieved by other processes through `output = yield Process` or `Process.value`
    - This output can be retrieved when the model has finished executing through `Process.value`.

